# Detecting & Clustering SWRs
Imports & path setup

In [4]:
%matplotlib 
%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt4Agg
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys
sys.path.append("/mnt/Data/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/")

In [6]:
import numpy as np
import matplotlib.pyplot as pl
import spectrum as sp
import pickle as pkl
import scipy.signal as scsig
import colormaps as mycmps
import signale.tools as tools
import os
import itertools
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
from matplotlib.mlab import PCA as mlabPCA
import matplotlib.gridspec as gridspec

/home/chenani/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/home/chenani/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [7]:
def zScore(array):
    return(array - np.average(array))/ np.std(array)
clPalette = sns.color_palette('deep')

In [8]:
animalPath = '/mnt/Data/ephysdata/Gerbils/'

In [9]:
lfpPaths = []
for lfp in tools.locate('*.lfp',animalPath):
    lfpPaths.append(os.path.join(lfp[0],lfp[1]))
lfpPaths = sorted(lfpPaths)
for path in lfpPaths:
    print path
NlfpFiles = len(lfpPaths)

/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-04_sleep1_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-04_sleep2_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-04_sleep3_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-05_sleep1_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-05_sleep2_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-05_sleep3_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-07_sleep1_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-07_sleep2_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-07_sleep3_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-09_sleep1_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-09_sleep2_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-09_sleep3_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-10_sleep1_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-10_sleep2_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-10_sleep3_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-14_sleep1_c

### Filtering & Whitening LFP signals

In [6]:
for path in lfpPaths:
    lfp = pkl.load(open(path,'rb'))
    if not hasattr(lfp,'signal_filtered'):
        lfp.filter(100,250)
        pkl.dump(lfp,open(path,'wb'),pkl.HIGHEST_PROTOCOL)
    if not hasattr(lfp,'signal_white'):
        lfp.whitenARMA()
        pkl.dump(lfp,open(path,'wb'),pkl.HIGHEST_PROTOCOL)

## Sleep epochs
Loading pre-identified sleep epochs and inserting them into neuralynxCSC objects and saving as .lfp files

In [7]:
SleepEpochs = pd.read_pickle(animalPath+'SleepEpochs.pd')
for ii,path in enumerate(lfpPaths):
    pickedChannel = pkl.load(open(path,'rb'))
    expDate = path.split('_')[0].split('/')[-1]
    session = path.split('_')[-2]
    session = session[:-1]+'0'+session[-1]
    relevantEpochs = SleepEpochs[np.logical_and(SleepEpochs['date']== expDate,
                                                SleepEpochs['session']==session)]
    relevantEpochs.index = range(relevantEpochs.index.size)
    Nepochs = relevantEpochs.shape[0]
    pickedChannel.rem_episodes = []
    pickedChannel.sws_episodes = []
    for idx in relevantEpochs.index:
        if relevantEpochs.loc[idx].epoch == 'REM':
            pickedChannel.rem_episodes.append([1000*relevantEpochs.loc[idx].t0,
                                               1000*relevantEpochs.loc[idx].t1])
        elif relevantEpochs.loc[idx].epoch == 'SWS':
            pickedChannel.sws_episodes.append([1000*relevantEpochs.loc[idx].t0,
                                               1000*relevantEpochs.loc[idx].t1])
    pickedChannel.rem_episodes = np.array(pickedChannel.rem_episodes)
    pickedChannel.sws_episodes = np.array(pickedChannel.sws_episodes)
    pkl.dump(pickedChannel,open(path,'wb'),pkl.HIGHEST_PROTOCOL)

## SWR Detection
Identify candidate SWR events and save for later!!!

In [129]:
for path in lfpPaths:
    lfp = pkl.load(open(path,'rb'))
    if lfp.rem_episodes.size and lfp.sws_episodes.size:
        lfp.hilbertTransform()
        if hasattr(lfp,'ripples') is True:
            print 'print processing %s' %path.split('/')[-1]
            lfp.ripple_recorder(sigma=12, length=12, rippleMix=False,rippleCut=True,SWRmix=True,removeREMripples = True)
            pkl.dump(lfp,open(path,'wb'),pkl.HIGHEST_PROTOCOL)

print processing 2011-08-05_sleep1_csc4.lfp
Ripple detection on CSC4.ncs  is DONE!!!
         Now you should see the attribute ripples with [t_start, t_end,peak value, t_peak] in each row for detected SWRs.
print processing 2011-08-05_sleep2_csc4.lfp
Ripple detection on CSC4.ncs  is DONE!!!
         Now you should see the attribute ripples with [t_start, t_end,peak value, t_peak] in each row for detected SWRs.
print processing 2011-08-05_sleep3_csc4.lfp
Ripple detection on CSC4.ncs  is DONE!!!
         Now you should see the attribute ripples with [t_start, t_end,peak value, t_peak] in each row for detected SWRs.
print processing 2011-08-07_sleep1_csc4.lfp
Ripple detection on CSC4.ncs  is DONE!!!
         Now you should see the attribute ripples with [t_start, t_end,peak value, t_peak] in each row for detected SWRs.
print processing 2011-08-07_sleep2_csc4.lfp
Ripple detection on CSC4.ncs  is DONE!!!
         Now you should see the attribute ripples with [t_start, t_end,peak value, t_pe

## SWR Dataframe
Building a pandas daraframe for candidate events!

In [219]:
rippDB = []
for path in lfpPaths[:]:
    lfp = pkl.load(open(path,'rb'))
    if lfp.rem_episodes.size and lfp.sws_episodes.size:
        expDate = path.split('_')[0].split('/')[-1]
        animal = path.split('/')[-3]
        session = path.split('_')[-2]
        session = session[:-1]+'0'+session[-1]
        rippDF = pd.DataFrame(lfp.ripples,columns=['t_start', 't_end','peak value', 't_peak'])
        rippDF['epoch'] = lfp.ripples.shape[0]*['NA']
        for item in lfp.sws_episodes:
            before = (rippDF.t_peak - lfp.timeAxis[0]) < item[1]
            after = (rippDF.t_peak - lfp.timeAxis[0]) > item[0]
            inBetween = np.logical_and(before,after)
            Nep = rippDF['epoch'][inBetween].size
            rippDF['epoch'][inBetween] = Nep * ['sws']
        for item in lfp.rem_episodes:
            before = (rippDF.t_peak - lfp.timeAxis[0]) < item[1]
            after = (rippDF.t_peak - lfp.timeAxis[0]) > item[0]
            inBetween = np.logical_and(before,after)
            Nep = rippDF['epoch'][inBetween].size
            rippDF['epoch'][inBetween] = Nep * ['rem']
        rippDF['date'] = rippDF.shape[0]*[expDate]
        rippDF['session'] = rippDF.shape[0]*[session]
        rippDF['animal'] = rippDF.shape[0]*[animal]
        rippDB.append(rippDF.as_matrix().flatten())
rippDB = np.hstack(rippDB)
rippDB = rippDB.reshape(rippDB.size/8,8)
rippDB = pd.DataFrame(rippDB,
                      columns=['t_i','t_f','peak_value','t_peak','epoch','date','session','animal'])

/home/chenani/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/chenani/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Adding more features to dataframe

In [240]:
HilbertAbsPeak = []
HilbertPhasePeak = []
WhitePeak = []
FilteredPeak = []
for path in lfpPaths[:]:
    pickedChannel = pkl.load(open(path,'rb'))
    expDate = path.split('_')[0].split('/')[-1]
    session = path.split('_')[-2]
    session = session[:-1]+'0'+session[-1]
    relevantEpochs = rippDB[np.logical_and(rippDB['date']== expDate,
                                                rippDB['session']==session)]
    indexArray = relevantEpochs.index
    for iii in indexArray:
        #rippDB.peak_value[iii]= (rippDB.peak_value[iii] - pickedChannel.signal.mean())/pickedChannel.signal.std()
        idx = tools.findNearest(pickedChannel.timeAxis,rippDB.t_peak[iii])[0]
        HilbertAbsPeak.append((pickedChannel.hilbertAbsSmooth[idx] - pickedChannel.hilbertAbsSmooth.mean())/pickedChannel.hilbertAbsSmooth.std())
        WhitePeak.append((pickedChannel.signal_white[idx] - pickedChannel.signal_white.mean())/pickedChannel.signal_white.std())
        HilbertPhasePeak.append(pickedChannel.hilbertPhase[idx])
        FilteredPeak.append((pickedChannel.signal_filtered[idx] - pickedChannel.signal_filtered.mean())/ pickedChannel.signal_filtered.std())
    #print path
rippDB['HilbertAbsPeak'] = np.array(HilbertAbsPeak)
rippDB['HilbertPhasePeak'] = np.array(HilbertPhasePeak)
rippDB['WhitePeak'] = np.array(WhitePeak)
rippDB['FilteredPeak'] = np.array(FilteredPeak)

/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-04_sleep1_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-04_sleep2_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-04_sleep3_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-05_sleep1_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-05_sleep2_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-05_sleep3_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-07_sleep1_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-07_sleep2_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-07_sleep3_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-09_sleep1_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-09_sleep2_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-09_sleep3_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-10_sleep1_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-10_sleep2_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-10_sleep3_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-14_sleep1_c

## Frequency and Signal Dataframes

In [52]:
rippPSD = []
whiteRippPSD = []
SWRsignal = []
for path in lfpPaths[:]:
    pickedChannel = pkl.load(open(path,'rb'))
    expDate = path.split('_')[0].split('/')[-1]
    session = path.split('_')[-2]
    session = session[:-1]+'0'+session[-1]
    relevantEpochs = rippDB[np.logical_and(rippDB['date']== expDate,
                                                rippDB['session']==session)]
    indexArray = relevantEpochs.index
    print path
    for idx in indexArray:
        tP = rippDB['t_peak'][idx]
        t1 = tools.findNearest(pickedChannel.timeAxis,tP-50)[0]
        t2 = tools.findNearest(pickedChannel.timeAxis,tP+50)[0]
        sig = pickedChannel.signal[t1:t2]
        #print sig
        wsig = pickedChannel.signal_white[t1:t2]
        p = sp.Periodogram(sig,sampling=2000,NFFT=256)
        wp = sp.Periodogram(wsig,sampling=2000,NFFT=256)
        p();
        wp();
        rippPSD.append(p.psd[:35])
        whiteRippPSD.append(wp.psd[:35])
        SWRsignal.append(sig)
rippPSD = np.array(rippPSD)
whiteRippPSD = np.array(whiteRippPSD)
SWRsignal = np.array(SWRsignal)

/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-04_sleep1_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-04_sleep2_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-04_sleep3_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-05_sleep1_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-05_sleep2_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-05_sleep3_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-07_sleep1_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-07_sleep2_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-07_sleep3_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-09_sleep1_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-09_sleep2_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-09_sleep3_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-10_sleep1_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-10_sleep2_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-10_sleep3_csc4.lfp
/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-14_sleep1_c

In [775]:
pkl.dump(rippPSD,open(animalPath+'rippPSD.npArr','wb'),pkl.HIGHEST_PROTOCOL)
pkl.dump(whiteRippPSD,open(animalPath+'whiteRippPSD.npArr','wb'),pkl.HIGHEST_PROTOCOL)
pkl.dump(SWRsignal,open(animalPath+'rippSignal.npArr','wb'),pkl.HIGHEST_PROTOCOL)

### Zcoring PSD dataframes

In [693]:
for jjj in range(rippPSD.shape[0]):
    rippPSD[jjj,:] = zScore(np.log10(rippPSD[jjj,:]))
    whiteRippPSD[jjj,:] = zScore(np.log10(whiteRippPSD[jjj,:]))

## PCA of PSD dataframes

Calculating PCA components of SWR power spectrum (both white and original) and adding the first 5 PC components to SWR dataframe!

In [349]:
pca = mlabPCA(rippPSD)
wpca = mlabPCA(whiteRippPSD)
usefulComps = pca.Y[:,range(5)]
wusefulComps = wpca.Y[:,range(5)]

In [357]:
rippDB['PC1'] = usefulComps[:,0]
rippDB['PC2'] = usefulComps[:,1]
rippDB['PC3'] = usefulComps[:,2]
rippDB['PC4'] = usefulComps[:,3]
rippDB['PC5'] = usefulComps[:,4]
rippDB['wPC1'] = wusefulComps[:,0]
rippDB['wPC2'] = wusefulComps[:,1]
rippDB['wPC3'] = wusefulComps[:,2]
rippDB['wPC4'] = wusefulComps[:,3]
rippDB['wPC5'] = wusefulComps[:,4]

In [20]:
rippDB.to_pickle(animalPath+'RipplesTimeDB.pd')

## Creating Feature files and Clustering

In [468]:
Rkeys =  rippDB.keys()
keys = Rkeys[[12,13,14,15,16]]
wkeys = Rkeys[[17,18,19,20,21]]
features = rippDB[keys].as_matrix()
wfeatures = rippDB[wkeys].as_matrix()

In [469]:
np.savetxt(animalPath+'pink.fet.1',features,fmt='%f',delimiter='\t',
           header=str(features.shape[1]),comments='')
np.savetxt(animalPath+'white.fet.1',wfeatures,fmt='%f',delimiter='\t',
           header=str(wfeatures.shape[1]),comments='')

Clustering is done using KlustaKwik program in unix console
## Loading Clusters

In [10]:
c = np.int16(np.loadtxt('/mnt/Data/ephysdata/Gerbils/pink.clu.1'))[1:]
wc = np.int16(np.loadtxt('/mnt/Data/ephysdata/Gerbils/white.clu.1'))[1:]

In [11]:
np.place(wc,wc==3,[1])
np.place(c,c==2,[1])
np.place(c,c==3,[2])

In [14]:
rippDB['wc'] = wc
rippDB['c'] = c

In [15]:
cCounts = np.histogram(c,bins=[0.5,1.5,2.5])[0]
wcCounts = np.histogram(wc,bins=[0.5,1.5,2.5])[0]
ccc = np.array([cCounts,wcCounts])

In [16]:
f,ax = pl.subplots(1,1)
ax.bar(np.array([1,2]), cCounts, 0.35,color= clPalette[0])
ax.bar(np.array([1,2]) + 0.35, wcCounts, 0.35,color= clPalette[1])
# add some text for labels, title and axes ticks
ax.set_ylabel('Number of Candidates')
ax.set_title('Clusters')
ax.set_xticks(np.array([1,2]) + 0.35)
ax.set_xticklabels(('C1', 'C2'))
ax.set_xlim([00.5,3])

(0.5, 3)

# PLOTS

In [15]:
rippDB = pd.read_pickle(animalPath+'RipplesTimeDB.pd')
rippPSD = pkl.load(open(animalPath+'rippPSD.npArr','rb'))
whiteRippPSD = pkl.load(open(animalPath+'whiteRippPSD.npArr','rb'))
SWRsignal = pkl.load(open(animalPath+'rippSignal.npArr','rb'))

In [18]:
A = np.logical_and(Rcond,(rippDB.epoch=='sws').as_matrix())
B = np.logical_and(Rcond,(rippDB.epoch=='rem').as_matrix())
C = np.logical_and(Gcond,(rippDB.epoch=='sws').as_matrix())
D = np.logical_and(Gcond,(rippDB.epoch=='rem').as_matrix())

In [19]:
#pl.figure()
sns.tsplot(SWRsignal[A.as_matrix(),:],time = np.arange(-50,50,0.5),color='b',ax=ax4)
sns.tsplot(SWRsignal[B.as_matrix(),:],time = np.arange(-50,50,0.5),color='r',ax=ax4)

IndexError: too many indices for array

In [20]:
pl.figure()
sns.tsplot(rippPSD[A.as_matrix(),:],time = p.frequencies()[:35],color='b')
sns.tsplot(rippPSD[B.as_matrix(),:],time = p.frequencies()[:35],color='r')
#sns.tsplot(rippPSD[C,:],time = p.frequencies()[:35],color='g')

NameError: name 'p' is not defined

In [28]:
pl.figure()
pl.scatter(rippDB.c+np.random.normal(0,0.1,14481),rippDB.wc++np.random.normal(0,0.1,14481),alpha=0.2)


In [23]:
pl.scatter(rippDB.PC2,rippDB.PC1,c=rippDB.c)

In [17]:
Rcond = np.logical_and(rippDB.c ==1, rippDB.wc == 1)
Gcond = np.logical_and(rippDB.c ==2, rippDB.wc == 2)
cond12 = np.logical_and(rippDB.c ==1, rippDB.wc == 2)
cond21 = np.logical_and(rippDB.c ==2, rippDB.wc == 1)

In [39]:
pl.figure()
sns.tsplot(rippPSD[Rcond.as_matrix(),:],time = p.frequencies()[:35],color='b')
sns.tsplot(rippPSD[Rcond.as_matrix(),:],time = p.frequencies()[:35],color='b',err_style="ci_bars", interpolate=False)
sns.tsplot(rippPSD[Gcond.as_matrix(),:],time = p.frequencies()[:35],color='r')
sns.tsplot(rippPSD[Gcond.as_matrix(),:],time = p.frequencies()[:35],color='r',err_style="ci_bars", interpolate=False)

In [70]:
pl.figure()
sns.tsplot(ss[Rcond.as_matrix(),:],time = np.arange(-50,50,0.5),c='b')
#sns.tsplot(ss[Rcond.as_matrix(),:],time = np.arange(-50,50,0.5),c='b',err_style="ci_bars", interpolate=False)
sns.tsplot(ss[Gcond.as_matrix(),:],time = np.arange(-50,50,0.5),c='r')
#sns.tsplot(ss[Gcond.as_matrix(),:],time = np.arange(-50,50,0.5),c='r',err_style="ci_bars", interpolate=False)

/home/chenani/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:892: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [49]:
rippPSD[1,:]

array([ 2.4397097 ,  2.08643027,  1.90360464,  1.69197734,  0.54431374,
       -0.29793649,  0.6502905 ,  0.8892712 ,  0.94758677,  0.3764311 ,
       -0.54343547,  0.25632259,  0.60104347,  0.26977032, -1.52393697,
       -0.28738658, -0.54081413, -0.84625668, -1.05377043, -0.93884031,
       -0.16703589,  0.31818468,  0.45181206,  0.41185459,  0.33045054,
        0.17144022, -0.15193115, -0.61699474, -1.08920116, -1.60333675,
       -1.28249608, -0.88301805, -0.74573892, -0.77748645, -0.99087749])

### Mauro
is great

In [62]:
SWRsignal.flatten().reshape(SWRsignal.size/200,200)

ValueError: total size of new array must be unchanged

In [67]:
ss = []
for iii,item in enumerate(SWRsignal):
    if item.size < 200:
        SWRsignal[iii] = np.append(item,np.zeros(200 - item.size))
        print iii
    ss.append(item)

In [69]:
ss = np.array(ss)